In [1]:
# Import necessary libraries
import json
import pandas as pd
import zipfile
import re

with open('config.json') as config_file:
    config = json.load(config_file)

# Extract values from the configuration
input_zip_path = config['clfs_zip_path']
excel_file_within_zip = config['clfs_excel_file']
output_path_template = config['clfs_output_path_template']
rate_column_prefix = config['rate_column_prefix']


# Open the ZIP file in read mode
with zipfile.ZipFile(input_zip_path, 'r') as z:
    # Open the specific Excel file within the ZIP file
    with z.open(excel_file_within_zip) as f:
        # Read the Excel file into a pandas DataFrame, skipping the first 4 rows
        df = pd.read_excel(f, skiprows=4)

# Use a regular expression to search for the pattern 'YYYY QxVx' in the file path
extracted_pattern = re.search(r'(\d{4} Q\dV\d)', excel_file_within_zip).group(0)

#Columns starting with 'RATE'
rate_columns = [col for col in df.columns if col.startswith(rate_column_prefix)]

# Select the required columns including the first 'RATE' column
df_selected = df[['HCPCS', 'EFF_DATE', rate_columns[0]]]

#output path for the selected columns in the dataframe
output_path = output_path_template.format(pattern=extracted_pattern)

# Export the DataFrame to an Excel workbook
df_selected.to_csv(output_path, index=False, sep='\t')

print("success")

KeyError: "There is no item named 'CLFS 2024 Q3V1.xlsx' in the archive"